In [82]:
import pickle
import os
import glob
import numpy as np
import json
from scipy.stats import norm

In [83]:
path = "/home/jenna/sacro_results/results/"

In [84]:
os.listdir(path)

['cf7cf863-929a-4b89-962b-7913869167e4',
 'a90983a1-2329-4019-96e8-949493c3fc24',
 '315ac662-9e95-4dec-80f0-48e244f9f8e1',
 'b71b28f1-7f63-4593-bdb2-9e60dd1ac71e',
 '96c75293-2026-48af-ad99-37281cb9bb3e',
 '436c714b-b588-4894-b68c-1e0f773e5df1',
 '5238ac7b-ad85-488f-b013-9ee4e2064e60',
 'e8d7e0c4-b293-43ce-832f-c37fb8fd3b23',
 'e6a9f8f7-8024-42d5-aad5-ef726d353f3b',
 '26d9e19b-59ed-4bbb-906a-9953b5d2c825',
 'a3347182-4364-43f8-8992-2409b1b66d31',
 'f63b25f4-8df4-4336-8c02-9456eb3c09b0',
 'slinwin_processor.py',
 'e8d3eabc-edd1-414f-9d95-277df742655a',
 'b6701cb1-3a9b-456e-ad73-c6e06eea03ec']

In [85]:
# Try one example of pred and true pickle file
example_path = path + os.listdir(path)[0]
example_path

'/home/jenna/sacro_results/results/cf7cf863-929a-4b89-962b-7913869167e4'

In [86]:
pred_labels = pickle.load(open(example_path + "/seq_pred.pickle", 'rb'))
e3d_pred_labels = pickle.load(open(example_path + "/seq_e3d.pickle", 'rb'))
mode_ave_pred_labels = pickle.load(open(example_path + "/seq_mode_av.pickle", 'rb'))
true_labels = pickle.load(open(example_path + "/seq_true.pickle", 'rb'))

In [87]:
print(pred_labels[:10])

[2, 4, 2, 5, 4, 4, 2, 4, 4, 2]


In [88]:
print(e3d_pred_labels[:10])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [89]:
print(mode_ave_pred_labels[:10])

[4.0, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]


In [90]:
print(true_labels[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [91]:
# scan through the true labels to see whether there are early phases repeating in the later stage
labels_list = []
labels_freq = []
for current_label in true_labels:
    #if list is empty or current label is not the same as the previous label, 
    #then append the current label and frequency
    if (not labels_list) or (current_label != labels_list[-1]):
        labels_list.append(current_label)
        labels_freq.append(1)
        continue
    # for the other cases, simply update frequency
    labels_freq[-1] += 1
    


In [92]:
# take a look at the frequencies of the labels
print("labels list:{}".format(labels_list))
print("labels freq:{}".format(labels_freq))

labels list:[1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5]
labels freq:[169, 8, 593, 12, 455, 1, 24, 6, 344, 9, 192]


_From the list and the frequencies, we can see that phase 3 has the highest proportion, and there are jumps from 3 to 4 and then back to 3. The time for the non-classed clips is the shortest between 4 and back to 3, probably because the tools for phase are is already there._

In [93]:
# Now, you need to loop through every video and document the label list and label freq
# Put above code into function
def get_label_list_and_freq(full_path):
#     pred_labels = pickle.load(open(full_path + "/seq_pred.pickle", 'rb'))
    true_labels = pickle.load(open(full_path + "/seq_true.pickle", 'rb'))
    # scan through the true labels to see whether there are early phases repeating in the later stage
    labels_list = []
    labels_freq = []
    for current_label in true_labels:
        #if list is empty or current label is not the same as the previous label, 
        #then append the current label and frequency
        if (not labels_list) or (current_label != labels_list[-1]):
            labels_list.append(current_label)
            labels_freq.append(1)
            continue
        # for the other cases, simply update frequency
        labels_freq[-1] += 1
    return (labels_list, labels_freq)

In [94]:
# Get the list of python files which are no subdirectories
py_files = glob.glob(path+"*.py")
for (i, file) in enumerate(py_files):
    file = file.replace(path, "")
    py_files[i] = file

In [95]:
py_files

['slinwin_processor.py']

In [96]:
# Remove the python file(s)
subdir_list = os.listdir(path)
for file in py_files:
    subdir_list.remove(file)
subdir_list

['cf7cf863-929a-4b89-962b-7913869167e4',
 'a90983a1-2329-4019-96e8-949493c3fc24',
 '315ac662-9e95-4dec-80f0-48e244f9f8e1',
 'b71b28f1-7f63-4593-bdb2-9e60dd1ac71e',
 '96c75293-2026-48af-ad99-37281cb9bb3e',
 '436c714b-b588-4894-b68c-1e0f773e5df1',
 '5238ac7b-ad85-488f-b013-9ee4e2064e60',
 'e8d7e0c4-b293-43ce-832f-c37fb8fd3b23',
 'e6a9f8f7-8024-42d5-aad5-ef726d353f3b',
 '26d9e19b-59ed-4bbb-906a-9953b5d2c825',
 'a3347182-4364-43f8-8992-2409b1b66d31',
 'f63b25f4-8df4-4336-8c02-9456eb3c09b0',
 'e8d3eabc-edd1-414f-9d95-277df742655a',
 'b6701cb1-3a9b-456e-ad73-c6e06eea03ec']

In [97]:
# Loop through the subdirectory and extract the label lists and frequencies
data_dict = {}
for subdir in subdir_list:
    full_path = path + subdir
    labels_list, labels_freq = get_label_list_and_freq(full_path)
    # store the list and corresponding frequency into a dictionary
    labels_dict = {}
    labels_dict["label list"] = labels_list
    labels_dict["label freq"] = labels_freq
    # store the label dictionary into the data dictionary under the corresponding subdirectory
    data_dict[subdir] = labels_dict

In [98]:
print(data_dict)

{'cf7cf863-929a-4b89-962b-7913869167e4': {'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5], 'label freq': [169, 8, 593, 12, 455, 1, 24, 6, 344, 9, 192]}, 'a90983a1-2329-4019-96e8-949493c3fc24': {'label list': [1, 0, 2, 0, 3, 0, 4, 0, 5], 'label freq': [41, 16, 334, 106, 476, 4, 7, 22, 120]}, '315ac662-9e95-4dec-80f0-48e244f9f8e1': {'label list': [1, 2, 0, 3, 0, 4, 0, 3, 0, 5], 'label freq': [84, 231, 23, 544, 24, 26, 39, 192, 7, 184]}, 'b71b28f1-7f63-4593-bdb2-9e60dd1ac71e': {'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5], 'label freq': [47, 3, 846, 1, 394, 3, 23, 10, 143, 2, 109]}, '96c75293-2026-48af-ad99-37281cb9bb3e': {'label list': [2, 1, 2, 0, 3, 0, 4, 0, 3, 0, 5], 'label freq': [146, 101, 254, 17, 550, 1, 73, 14, 190, 15, 266]}, '436c714b-b588-4894-b68c-1e0f773e5df1': {'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5], 'label freq': [153, 45, 348, 39, 346, 2, 51, 17, 221, 10, 131]}, '5238ac7b-ad85-488f-b013-9ee4e2064e60': {'label list': [1, 0, 2, 0, 3, 0, 4, 0, 5], 'label freq': [

Results are as shown:  

{'cf7cf863-929a-4b89-962b-7913869167e4':   
    {  
    'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5],   
    'label freq': [169, 8, 593, 12, 455, 1, 24, 6, 344, 9, 192]  
     },   
 'a90983a1-2329-4019-96e8-949493c3fc24':   
     {  
     'label list': [1, 0, 2, 0, 3, 0, 4, 0, 5],   
     'label freq': [41, 16, 334, 106, 476, 4, 7, 22, 120]  
     },   
 '315ac662-9e95-4dec-80f0-48e244f9f8e1':   
     {  
     'label list': [1, 2, 0, 3, 0, 4, 0, 3, 0, 5],      
     'label freq': [84, 231, 23, 544, 24, 26, 39, 192, 7, 184]  
     },   
     
 'b71b28f1-7f63-4593-bdb2-9e60dd1ac71e':   
     {  
     'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5],   
     'label freq': [47, 3, 846, 1, 394, 3, 23, 10, 143, 2, 109]  
     },   
 '96c75293-2026-48af-ad99-37281cb9bb3e':   
     {  
     'label list': [2, 1, 2, 0, 3, 0, 4, 0, 3, 0, 5],   
     'label freq': [146, 101, 254, 17, 550, 1, 73, 14, 190, 15, 266]  
     },   
 '436c714b-b588-4894-b68c-1e0f773e5df1':   
     {  
     'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5],   
     'label freq': [153, 45, 348, 39, 346, 2, 51, 17, 221, 10, 131]  
     },   
 '5238ac7b-ad85-488f-b013-9ee4e2064e60':   
     {  
     'label list': [1, 0, 2, 0, 3, 0, 4, 0, 5],   
     'label freq': [54, 1, 728, 28, 379, 14, 21, 62, 151]  
     },   
 'e8d7e0c4-b293-43ce-832f-c37fb8fd3b23':   
     {
     'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5],   
     'label freq': [125, 1, 257, 22, 247, 2, 32, 3, 48, 5, 133]  
     }, 
 'e6a9f8f7-8024-42d5-aad5-ef726d353f3b':   
     {  
     'label list': [1, 2, 0, 3, 4, 0, 5],   
     'label freq': [67, 649, 70, 606, 42, 57, 241]  
     },   
 '26d9e19b-59ed-4bbb-906a-9953b5d2c825':   
     {  
     'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5],  
     'label freq': [69, 1, 334, 43, 428, 2, 6, 5, 193, 14, 207]  
     },   
 'a3347182-4364-43f8-8992-2409b1b66d31':   
     {  
     'label list': [2, 0, 1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5],  
     'label freq': [52, 2, 7, 1, 85, 20, 455, 1, 15, 6, 46, 20, 213]  
     },    
 'f63b25f4-8df4-4336-8c02-9456eb3c09b0':   
     {  
     'label list': [1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5],   
     'label freq': [202, 4, 450, 23, 434, 8, 23, 3, 301, 10, 107]  
     },   
 'e8d3eabc-edd1-414f-9d95-277df742655a':    
     {  
     'label list': [2, 0, 1, 0, 2, 0, 3, 0, 4, 0, 3],   
     'label freq': [152, 1, 68, 2, 316, 69, 418, 1, 9, 19, 54]  
     },  
 'b6701cb1-3a9b-456e-ad73-c6e06eea03ec':  
     {  
     'label list': [1, 2, 0, 3, 4, 0, 3, 0, 5],   
     'label freq': [137, 741, 30, 218, 93, 9, 253, 50, 119]  
     }  
 }  
 

In [99]:
print("Number of videos:{}".format(len(data_dict)))

Number of videos:14


### Pattern observed:
1. have 2 -> 1 -> 2 pattern, starting with phase 2 as the initial phase, BUT there isn't 1 -> 2 -> 1. Hence may need memory to flag whether the current phase 2 is the starting phase, coz if it is, then it has probability of jumping back to phase 1, otherwise it will only go to phase 3.
2. have 3 -> 4 -> 3 pattern, but the ending 3 is usually followed by 5. Sometimes it ends at the ending 3. 

In [100]:
# Split the data into the ones with repeating phases and the ones without
# function returns boolean value indicating whether the current video contains non-zero repeating phases
def is_repeating(data):
    '''
    data: the dict keyed using video sequence number
    '''
    prev_phases = []
    for cur_phase in data["label list"]:
        # don't count zeros for repeating phases
        if cur_phase == 0: continue
        # if current phase has been witnessed, return true
        if cur_phase in prev_phases:
            return True
        # else, add the current phase into the prev_phases
        prev_phases.append(cur_phase)
    # return false if no repeating phase
    return False
        
        

In [101]:
# Loop through all data item and split the videos into two lists
no_repeating = []
has_repeating = []
for vid in data_dict:
    if is_repeating(data_dict[vid]):
        has_repeating.append(vid)
    else:
        no_repeating.append(vid)

In [102]:
print("There are {} videos with repeating phases:{}".format(len(has_repeating), has_repeating))
print()
print("There are {} videos without repeating phases:{}".format(len(no_repeating), no_repeating))

There are 11 videos with repeating phases:['cf7cf863-929a-4b89-962b-7913869167e4', '315ac662-9e95-4dec-80f0-48e244f9f8e1', 'b71b28f1-7f63-4593-bdb2-9e60dd1ac71e', '96c75293-2026-48af-ad99-37281cb9bb3e', '436c714b-b588-4894-b68c-1e0f773e5df1', 'e8d7e0c4-b293-43ce-832f-c37fb8fd3b23', '26d9e19b-59ed-4bbb-906a-9953b5d2c825', 'a3347182-4364-43f8-8992-2409b1b66d31', 'f63b25f4-8df4-4336-8c02-9456eb3c09b0', 'e8d3eabc-edd1-414f-9d95-277df742655a', 'b6701cb1-3a9b-456e-ad73-c6e06eea03ec']

There are 3 videos without repeating phases:['a90983a1-2329-4019-96e8-949493c3fc24', '5238ac7b-ad85-488f-b013-9ee4e2064e60', 'e6a9f8f7-8024-42d5-aad5-ef726d353f3b']


# Get probability vectors list

In [103]:
# now that I have got the probability distribution I can try PKI to see whether there is improvement
prob_vec_path = '/home/jenna/jennaCode/model_outputs' 
os.listdir(prob_vec_path)

['cf7cf863-929a-4b89-962b-7913869167e4',
 'a90983a1-2329-4019-96e8-949493c3fc24',
 '.ipynb_checkpoints',
 '315ac662-9e95-4dec-80f0-48e244f9f8e1',
 'b71b28f1-7f63-4593-bdb2-9e60dd1ac71e',
 '96c75293-2026-48af-ad99-37281cb9bb3e',
 '436c714b-b588-4894-b68c-1e0f773e5df1',
 '5238ac7b-ad85-488f-b013-9ee4e2064e60',
 'e8d7e0c4-b293-43ce-832f-c37fb8fd3b23',
 'e6a9f8f7-8024-42d5-aad5-ef726d353f3b',
 '26d9e19b-59ed-4bbb-906a-9953b5d2c825',
 'a3347182-4364-43f8-8992-2409b1b66d31',
 'f63b25f4-8df4-4336-8c02-9456eb3c09b0',
 'e8d3eabc-edd1-414f-9d95-277df742655a',
 'b6701cb1-3a9b-456e-ad73-c6e06eea03ec']

In [104]:
# Try one example of prob vec pickle file
example_prob_vec_path = os.path.join(prob_vec_path, os.listdir(prob_vec_path)[0])
example_prob_vec_path

'/home/jenna/jennaCode/model_outputs/cf7cf863-929a-4b89-962b-7913869167e4'

In [105]:
example_prob_vec = pickle.load(open(example_prob_vec_path + "/prob_vectors.pickle", 'rb'))
example_pred_labels = pickle.load(open(example_prob_vec_path + "/seq_pred.pickle", 'rb'))

In [106]:
print(example_prob_vec[:10])

[[0.01613  0.206004 0.750325 0.003535 0.013024 0.010982]
 [0.066015 0.018182 0.202156 0.069131 0.602181 0.042336]
 [0.048635 0.052382 0.518992 0.063645 0.259962 0.056384]
 [0.101346 0.025392 0.493788 0.003663 0.342448 0.033362]
 [0.027749 0.006123 0.012191 0.012926 0.937361 0.003651]
 [0.015823 0.000046 0.00195  0.004879 0.976102 0.001202]
 [0.008318 0.705869 0.277029 0.000294 0.001178 0.007312]
 [0.108517 0.011892 0.074766 0.088726 0.477993 0.238106]
 [0.042795 0.003294 0.019624 0.036435 0.884503 0.013348]
 [0.002738 0.002613 0.955466 0.002455 0.03586  0.000868]]


In [107]:
# pred labels from data given by yitong
print(pred_labels[:10])

[2, 4, 2, 5, 4, 4, 2, 4, 4, 2]


In [108]:
pred_from_prob = [np.argmax(arr) for arr in example_prob_vec]

In [109]:
 print(pred_from_prob[:500])

[2, 4, 2, 2, 4, 4, 1, 4, 4, 2, 2, 5, 2, 5, 1, 1, 5, 5, 1, 1, 1, 1, 1, 1, 5, 5, 5, 1, 1, 1, 1, 2, 1, 1, 1, 2, 4, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 4, 5, 5, 4, 4, 3, 5, 5, 5, 5, 1, 1, 1, 1, 2, 5, 1, 1, 1, 5, 5, 5, 5, 1, 4, 4, 1, 5, 1, 4, 1, 1, 5, 1, 1, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 1, 2, 1, 2, 5, 2, 2, 2, 5, 5, 5, 2, 2, 2, 2, 5, 2, 2, 2, 5, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 4, 2, 1, 2, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 5, 2, 1, 1, 5, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 5, 2, 5, 3, 5, 2, 3, 2, 5, 2, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 4, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 5, 3, 2, 2, 2, 2, 4, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 

In [110]:
print(type(pred_labels))

<class 'list'>


In [111]:
# accuracy
from sklearn.metrics import accuracy_score, precision_score

accuracy_score(true_labels, pred_labels)

0.6530612244897959

In [112]:
# try implement PKI and check for accuracy again
'''
Prior Knowledge Inference:
Use prior knowledge of the phases to determine the current phase
Threshold 1: number of phases to update the prior phase - set to 5 for now 
Threshold 2: probability threshold to keep the results from CNN - set to 0.5 for now, i.e. no bias towards or against CNN
Prior phase: initialize to phase 1 for now
'''
def pki(prob_vec_list, seq_pred, prior_th, prob_th):
    pred_output = np.array(seq_pred).copy()
    # initialise prior phase to phase 1
    prior = 1
    prev = pred_output[0]
    # keep a counter to count how many new phases have appeared. if count > 5, update the prior to new phases
    count_new_phases = 0
    for index, prediction in enumerate(pred_output):
#         print("Current prior is {}".format(prior))
#         print("Count of prediction {} is {}".format(prev, count_new_phases))
        # if counter > 5, update prior
        if count_new_phases > prior_th:
            prior = prev
        # if CNN probability is bigger than 0.5, keep CNN prediction; else update it to prior
        if prob_vec_list[index][prediction] < prob_th:
            pred_output[index] = prior
        # if current prediction is same as the previous one, counter increment by 1
        if prediction == prev:
            count_new_phases += 1
        else:
            # reset counter
            prev = prediction
            count_new_phases = 0
    
    return pred_output
        
    

In [113]:
pki_output = pki(example_prob_vec, pred_labels, 4, 1)

In [114]:
print(pki_output[:500])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [115]:
print("Accuracy of using only CNN for prediction is {}".format(accuracy_score(true_labels, pred_labels)))
print("Accuracy of using CNN followed by PKI for prediction is {}".format(accuracy_score(true_labels, pki_output)))

Accuracy of using only CNN for prediction is 0.6530612244897959
Accuracy of using CNN followed by PKI for prediction is 0.809707666850524


## record results here


accuracy of using 3 and 0.5: 0.7352454495311638   
accuracy of using 5 and 0.5: 0.7380033094318809  
accuracy of using 8 and 0.5: 0.7363485934914507  
accuracy of using 10 and 0.5: 0.7357970215113072   

accuracy of using 5 and 0.6: 0.758411472697187   
accuracy of using 5 and 0.7: 0.7716492002206288   
accuracy of using 5 and 0.75: 0.7793712079426365   
accuracy of using 5 and 0.78: 0.7815774958632101  
accuracy of using 5 and 0.8: 0.7848869277440707    
accuracy of using 5 and 1: 0.8080529509100938   

accuracy of using 4 and 0.95: 0.797573083287369  
accuracy of using 5 and 0.95: 0.7931605074462217  

accuracy of using 3 and 1: 0.8041919470490899  
accuracy of using 4 and 1: 0.809707666850524   
accuracy of using 5 and 1: 0.8080529509100938  
accuracy of using 6 and 1: 0.8019856591285163 

**The results show that when disregarding the probability from CNN, and completely depending on prior, it gives the best performance.**

**Problem: no way to detect zero phase based on CNN results**

# Get PKI results for all the videos based on fold and division

In [35]:
# hand tune for each fold. change the parameters here#######
index = 6
prob_th = 1
##############################


# ALL PATHS
paramPath = "/home/yitong/venv_yitong/sacro_wf_analysis/params"
path_prefix= '/home/yitong/venv_yitong/sacro_wf_analysis/'
whole_path = path_prefix + 'data/sacro_sequence/whole'
videos_path = path_prefix + 'data/sacro_jpg'
output_path = '/home/jenna/jennaCode/model_outputs'
true_label_path = "/home/jenna/sacro_results/results/"

# folds for cross vaildation devision
folders = ['folder1', 'folder2', 'folder3', 'folder4', 'folder5', 'folder6', 'folder7']
divs = ['div1', 'div2', 'div3', 'div4', 'div5', 'div6', 'div7']


folder = folders[index]
div = divs[index]

whole_folder_path = os.path.join(whole_path, folder)
with open(os.path.join(videos_path, 'dataset_' + div + '.json'), 'r') as json_data:
    temp = json.load(json_data)
# use all training and validation data for training
video_train = temp['train']
video_validation = temp['validation']
print('The current training video train: {}'.format(video_train))
video_test = temp['test']
print('The current testing video train: {}'.format(video_test))

# apply pki with the video_train to get an average accuracy
# tune parameters with different prior threshold
for prior_th in [1,2,3,4,5,6,7,8,9,10]:
    ave_tr_acc = 0
    for video in video_train:
        prob_vec = pickle.load(open(output_path + '/' + video + "/prob_vectors.pickle", 'rb'))
        pred_labels = pickle.load(open(output_path + '/' + video + "/seq_pred.pickle", 'rb'))
        true_labels = pickle.load(open(true_label_path +  '/' + video + "/seq_true.pickle", 'rb'))
        pki_output = pki(prob_vec, pred_labels, prior_th, prob_th)
        acc = accuracy_score(true_labels, pki_output)
        ave_tr_acc += acc

#         print("Video {} has training accuracy {}".format(video, acc))
    #     print("True labels: {}".format(true_labels[:20]))
    #     print("Pred labels: {}".format(pred_labels[:20]))
    #     print("PKI outputs: {}".format(pki_output[:20]))

    ave_tr_acc /= len(video_train)
    print("Average accuracy of fold {} with prior_th = {} and prob_th = {} is {}".format(index+1, prior_th, prob_th, ave_tr_acc))


The current training video train: ['5238ac7b-ad85-488f-b013-9ee4e2064e60', '436c714b-b588-4894-b68c-1e0f773e5df1', 'b71b28f1-7f63-4593-bdb2-9e60dd1ac71e', 'a3347182-4364-43f8-8992-2409b1b66d31', 'e6a9f8f7-8024-42d5-aad5-ef726d353f3b', '96c75293-2026-48af-ad99-37281cb9bb3e', '315ac662-9e95-4dec-80f0-48e244f9f8e1', 'a90983a1-2329-4019-96e8-949493c3fc24', 'cf7cf863-929a-4b89-962b-7913869167e4', 'b6701cb1-3a9b-456e-ad73-c6e06eea03ec']
The current testing video train: ['e8d3eabc-edd1-414f-9d95-277df742655a', '26d9e19b-59ed-4bbb-906a-9953b5d2c825']
Average accuracy of fold 7 with prior_th = 1 and prob_th = 1 is 0.7211315186687357
Average accuracy of fold 7 with prior_th = 2 and prob_th = 1 is 0.745530433993306
Average accuracy of fold 7 with prior_th = 3 and prob_th = 1 is 0.7619634588309347
Average accuracy of fold 7 with prior_th = 4 and prob_th = 1 is 0.7554538678552799
Average accuracy of fold 7 with prior_th = 5 and prob_th = 1 is 0.7426100629819942
Average accuracy of fold 7 with prior

In [36]:
# prior_th = 6
prob_th = 1

# apply pki with the same parameters on validation data
for prior_th in [1,2,3,4,5,6,7,8,9,10]:
    ave_val_acc = 0
    for video in video_validation:
        prob_vec = pickle.load(open(output_path + '/' + video + "/prob_vectors.pickle", 'rb'))
        pred_labels = pickle.load(open(output_path + '/' + video + "/seq_pred.pickle", 'rb'))
        true_labels = pickle.load(open(true_label_path +  '/' + video + "/seq_true.pickle", 'rb'))
        pki_output = pki(prob_vec, pred_labels, prior_th, prob_th)
#         print("Prediction accuracy by CNN: {} ".format(accuracy_score(true_labels, pred_labels)))
        acc = accuracy_score(true_labels, pki_output)
#         precision = precision_score(true_labels, pki_output, labels=[0,1,2,3,4,5], average='macro')
        ave_val_acc += acc

#         print("Video {} with prior_th = {} has validation accuracy {} and precision {}".format(video, prior_th, acc, precision))
#         print("True labels: {}".format(true_labels[:100]))
#         print("Pred labels: {}".format(pred_labels[:100]))
#         print("PKI outputs: {}".format(pki_output[:100]))

    ave_val_acc /= len(video_validation)
    print("Average validation accuracy of fold {} with prior_th = {} and prob_th = {} is {}".format(index+1, prior_th, prob_th, ave_val_acc))


Average validation accuracy of fold 7 with prior_th = 1 and prob_th = 1 is 0.7666216339570973
Average validation accuracy of fold 7 with prior_th = 2 and prob_th = 1 is 0.7906636239160201
Average validation accuracy of fold 7 with prior_th = 3 and prob_th = 1 is 0.8061159287996349
Average validation accuracy of fold 7 with prior_th = 4 and prob_th = 1 is 0.7744554997717936
Average validation accuracy of fold 7 with prior_th = 5 and prob_th = 1 is 0.7700009128251939
Average validation accuracy of fold 7 with prior_th = 6 and prob_th = 1 is 0.756403468735737
Average validation accuracy of fold 7 with prior_th = 7 and prob_th = 1 is 0.7291172980374259
Average validation accuracy of fold 7 with prior_th = 8 and prob_th = 1 is 0.7395198539479689
Average validation accuracy of fold 7 with prior_th = 9 and prob_th = 1 is 0.7373509812870835
Average validation accuracy of fold 7 with prior_th = 10 and prob_th = 1 is 0.735182108626198


In [37]:
(0.8362066414489024 + 0.7426324701757324)/2

0.7894195558123174

In [38]:
(0.7725121287694008 + 0.7887248634434596)/2

0.7806184961064302

In [39]:
# finally report evaluation metrics on testing data
prior_th = 3
ave_test_acc = 0
for video in video_test:
    prob_vec = pickle.load(open(output_path + '/' + video + "/prob_vectors.pickle", 'rb'))
    pred_labels = pickle.load(open(output_path + '/' + video + "/seq_pred.pickle", 'rb'))
    true_labels = pickle.load(open(true_label_path +  '/' + video + "/seq_true.pickle", 'rb'))
    pki_output = pki(prob_vec, pred_labels, prior_th, prob_th)
    print("Prediction accuracy by CNN: {} ".format(accuracy_score(true_labels, pred_labels)))
    acc = accuracy_score(true_labels, pki_output)
    ave_test_acc += acc
    
    print("Video {} has test accuracy {}".format(video, acc))
    print("True labels: {}".format(true_labels[:100]))
    print("Pred labels: {}".format(pred_labels[:100]))
    print("PKI outputs: {}".format(pki_output[:100]))
    
ave_test_acc /= len(video_test)
print("Average test accuracy of fold {} with prior_th = {} and prob_th = {} is {}".format(index+1, prior_th, prob_th, ave_test_acc))

Prediction accuracy by CNN: 0.7069431920649234 
Video e8d3eabc-edd1-414f-9d95-277df742655a has test accuracy 0.8295761947700632
True labels: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Pred labels: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 4, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2]
PKI outputs: [1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Prediction accuracy by CNN:

## record results of training PKI on all folds:
Average accuracy of fold 1 with prior_th = 4 and prob_th = 1 is 0.7574578705998676  
Average accuracy of fold 1 with prior_th = 4 and prob_th = 0.8 is 0.725925655636102  
Average accuracy of fold 1 with prior_th = 4 and prob_th = 0.6 is 0.6904438413378784   

Average accuracy of fold 1 with prior_th = 3 and prob_th = 1 is 0.7668683290478187   

Average accuracy of fold 1 with prior_th = 2 and prob_th = 1 is 0.7498360932292092   

=======
fold 1: prior_th = 3 and prob_th = 1
=======  


Average accuracy of fold 2 with prior_th = 2 and prob_th = 1 is 0.7599713052156475  
Average accuracy of fold 2 with prior_th = 3 and prob_th = 1 is 0.7790179193036015  
Average accuracy of fold 2 with prior_th = 4 and prob_th = 1 is 0.7599173883204134  

Average accuracy of fold 2 with prior_th = 2 and prob_th = 0.9 is 0.7420986687927811  
Average accuracy of fold 2 with prior_th = 3 and prob_th = 0.9 is 0.7553859703305618  
Average accuracy of fold 2 with prior_th = 4 and prob_th = 0.9 is 0.7455247430460634  

=======
fold 2: prior_th = 3 and prob_th = 1
=======  


Average accuracy of fold 3 with prior_th = 3 and prob_th = 1 is 0.7959578556941407   
Average accuracy of fold 3 with prior_th = 3 and prob_th = 0.9 is 0.7768489720322854  

=======
fold 3: prior_th = 3 and prob_th = 1
=======  


Average accuracy of fold 4 with prior_th = 3 and prob_th = 1 is 0.7891135666223035    
Average accuracy of fold 4 with prior_th = 4 and prob_th = 1 is 0.7932076436941349   
Average accuracy of fold 4 with prior_th = 5 and prob_th = 1 is 0.7804359841164158   

Training data with 4 has higher accuracy BUT validation data with 3 has higher accuracy, so I used 3.

=======
fold 4: prior_th = 3 and prob_th = 1
=======  

Average accuracy of fold 5 with prior_th = 3 and prob_th = 1 is 0.7725121287694008  
Average accuracy of fold 5 with prior_th = 4 and prob_th = 1 is 0.7596752493690193  
Average accuracy of fold 5 with prior_th = 5 and prob_th = 1 is 0.7407984014368268  
Average accuracy of fold 5 with prior_th = 6 and prob_th = 1 is 0.7426324701757324  

Average validation accuracy of fold 5 with prior_th = 3 and prob_th = 1 is 0.7887248634434596  
Average validation accuracy of fold 5 with prior_th = 4 and prob_th = 1 is 0.8095614068206332  
Average validation accuracy of fold 5 with prior_th = 5 and prob_th = 1 is 0.8286145263709991  
Average validation accuracy of fold 5 with prior_th = 6 and prob_th = 1 is 0.8362066414489024  

=======
fold 5: prior_th = 6 and prob_th = 1
=======  

=======
fold 6: prior_th = 3 and prob_th = 1
=======  

=======
fold 7: prior_th = 3 and prob_th = 1
=======  



----------

>Conclusion: softmax layer output doesn't really make any impact. The results are the best when PKI completely depends on prediction 

### report evaluation of trained PKI prior parameters on validation and testing data for all folds
fold 1:  
Average validation accuracy of fold 1 with prior_th = 3 and prob_th = 1 is 0.7904173561316419   
Average test accuracy of fold 1 with prior_th = 3 and prob_th = 1 is 0.8061159287996349   

fold 2:  
Average validation accuracy of fold 2 with prior_th = 3 and prob_th = 1 is 0.7453679775207201  
Average test accuracy of fold 2 with prior_th = 3 and prob_th = 1 is 0.7904173561316419  

fold 3:  
Average validation accuracy of fold 2 with prior_th = 3 and prob_th = 1 is 0.7057176741789462  
Average test accuracy of fold 2 with prior_th = 3 and prob_th = 1 is 0.7453679775207201  

fold 4:  
Average validation accuracy of fold 4 with prior_th = 3 and prob_th = 1 is 0.7795894228799058  
Average test accuracy of fold 4 with prior_th = 3 and prob_th = 1 is 0.7057176741789462

fold 5:  
Average validation accuracy of fold 5 with prior_th = 6 and prob_th = 1 is 0.8362066414489024  
Average test accuracy of fold 5 with prior_th = 6 and prob_th = 1 is 0.7452645616314537  

fold 6:  
Average validation accuracy of fold 6 with prior_th = 3 and prob_th = 1 is 0.8149417072160607  
Average test accuracy of fold 6 with prior_th = 3 and prob_th = 1 is 0.7887248634434596  


fold 7:  
Average accuracy of fold 7 with prior_th = 3 and prob_th = 1 is 0.7619634588309347  
Average test accuracy of fold 7 with prior_th = 3 and prob_th = 1 is 0.8149417072160607   

In [40]:
ave_test = 0.8061159287996349 + 0.7904173561316419 + 0.7453679775207201 + 0.7057176741789462 + 0.7452645616314537  + 0.7887248634434596  + 0.8149417072160607   
ave_test /= 7
print("average testing accuracy of using vanilla PKI on CNN prediction results: {}".format(ave_test))

average testing accuracy of using vanilla PKI on CNN prediction results: 0.7709357241317026


# Get other prior statistics from ground truth

### Other priors that can be considered
- number of phases into the current prior (i.e. number of phases into the current phase), which can then be used to change the phase prior accordingly   -> current incremental interval
- probability of being in the current phase -> the number of total current phases divided by total number of phases, which can be used to change the weight (which can be used to determine the probability of the current prior phase)
- transition probability from state i to j= (the most primitive:) number of transitions from i to j devided by all transitions from i (should be fixed)
- Therefore, the overall probability that the transition from State 0 to State 1 will occur during a specific incremental interval of time dt is given by multiplying (1) the probability of being in State 0 at the beginning of that interval (based on percentage?), and (2) the probability of the transition during an interval dt given that it was in State 0 at the beginning of that increment. 
- There is a transition probability, which is the rate parameter, which changes the probability of staying in the current phase for each time
- Based on the exponential modeling, P_0(t) = e^(-lambda \times t),  P_1(t) = 1 - e^(-lambda \times t)

## Prior parameters:

In [41]:
for vid in data_dict:
    print("{}:".format(vid))
    print("label list:{}\nlabel freq:{}".format(data_dict[vid]["label list"], data_dict[vid]["label freq"]))

cf7cf863-929a-4b89-962b-7913869167e4:
label list:[1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5]
label freq:[169, 8, 593, 12, 455, 1, 24, 6, 344, 9, 192]
a90983a1-2329-4019-96e8-949493c3fc24:
label list:[1, 0, 2, 0, 3, 0, 4, 0, 5]
label freq:[41, 16, 334, 106, 476, 4, 7, 22, 120]
315ac662-9e95-4dec-80f0-48e244f9f8e1:
label list:[1, 2, 0, 3, 0, 4, 0, 3, 0, 5]
label freq:[84, 231, 23, 544, 24, 26, 39, 192, 7, 184]
b71b28f1-7f63-4593-bdb2-9e60dd1ac71e:
label list:[1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5]
label freq:[47, 3, 846, 1, 394, 3, 23, 10, 143, 2, 109]
96c75293-2026-48af-ad99-37281cb9bb3e:
label list:[2, 1, 2, 0, 3, 0, 4, 0, 3, 0, 5]
label freq:[146, 101, 254, 17, 550, 1, 73, 14, 190, 15, 266]
436c714b-b588-4894-b68c-1e0f773e5df1:
label list:[1, 0, 2, 0, 3, 0, 4, 0, 3, 0, 5]
label freq:[153, 45, 348, 39, 346, 2, 51, 17, 221, 10, 131]
5238ac7b-ad85-488f-b013-9ee4e2064e60:
label list:[1, 0, 2, 0, 3, 0, 4, 0, 5]
label freq:[54, 1, 728, 28, 379, 14, 21, 62, 151]
e8d7e0c4-b293-43ce-832f-c37fb8fd3b23:
label l

## Initialisation Probability

In [42]:
# Initialisation probability
def getInitProb(data_dict):
    # initialise 
    init_count = np.array([0]*6)
    for vid in data_dict:
        init_phase = data_dict[vid]['label list'][0]
        init_count[init_phase] += 1
    print("Count of initial phases: {}".format(init_count))
    total = np.sum(init_count)
    print("Total:{}".format(total))
    
    init_prob = np.array([0.0]*6)
    for i in range(len(init_prob)):
        init_prob[i] = init_count[i]/total
    print("Initialisation probability vector: {}".format(init_prob))
    return init_prob

In [43]:
init_prob = getInitProb(data_dict)

Count of initial phases: [ 0 11  3  0  0  0]
Total:14
Initialisation probability vector: [0.         0.78571429 0.21428571 0.         0.         0.        ]


In [44]:
# rearrange the probability based on size and store the phase accordingly for use later
init_prob_sorted = np.sort(init_prob)
non_zero_init_prob_sorted = []
# disregard all the zeros
for prob in init_prob_sorted:
    if prob != 0:
        non_zero_init_prob_sorted.append(prob)
non_zero_init_prob_sorted = np.array(non_zero_init_prob_sorted)
print(non_zero_init_prob_sorted)

[0.21428571 0.78571429]


In [45]:
# loop through unsorted init prob vector and find the corresponding phases
non_zero_init_prob_phase_num = len(non_zero_init_prob_sorted)
init_phases_sorted = [0]*non_zero_init_prob_phase_num
for phase, prob in enumerate(init_prob):
    if prob == 0:
        continue
    # check the phase of the corresponding non-zero init prob
    for index in range(non_zero_init_prob_phase_num):
        if non_zero_init_prob_sorted[index] == prob:
            init_phases_sorted[index] = phase
            
            
print("Corresponding phases with order sorted in initialisation probability:")
print(init_phases_sorted)

Corresponding phases with order sorted in initialisation probability:
[2, 1]


## Mean and Standard Devidation of the percentage length of each phase 

In [46]:
# Get phase duration percentage for each video
def getPhasePercByVid(data_dict):
    # dictionary to store all percentage vectors
    phase_perc_mat = np.array([[0,0,0,0,0,0]])
    for vid in data_dict:
        # init a counter for every video
        phase_count = np.array([0]*6)
        # get total frame in the end
        total = 0
        
        for index, freq in enumerate(data_dict[vid]['label freq']):
            # get the corresponding label and add the counts to the label
            phase_count[data_dict[vid]['label list'][index]] += freq
            total += freq
        
        # get the percentage for each phase
        phase_perc = np.array([0.0]*6)
        for i in range(6):
            phase_perc[i] = phase_count[i]/total
            
        # add the vector to the dictionary
        phase_perc_mat = np.append(phase_perc_mat, np.array([phase_perc]), axis=0)
    
    # remove the first array that gives the shape of the numpy array
    return np.delete(phase_perc_mat, 0, axis=0)

In [47]:
phase_perc_mat_by_vid = getPhasePercByVid(data_dict)
print("Phase percentage matrix by video:")
print(phase_perc_mat_by_vid)
print("Shape of matrix:{}".format(phase_perc_mat_by_vid.shape))
print("Total number of videos:{}".format(len(phase_perc_mat_by_vid)))

Phase percentage matrix by video:
[[0.01985659 0.09321566 0.32708218 0.44070601 0.01323773 0.10590182]
 [0.13143872 0.03641208 0.29662522 0.42273535 0.0062167  0.10657194]
 [0.06868538 0.0620384  0.17060561 0.54357459 0.01920236 0.13589365]
 [0.01201771 0.02972802 0.53510436 0.33965844 0.01454775 0.06894371]
 [0.02888752 0.06207744 0.24585126 0.45482483 0.04486785 0.16349109]
 [0.08290536 0.11225238 0.25531915 0.41599413 0.03741746 0.09611152]
 [0.07301808 0.03755216 0.50625869 0.2635605  0.01460362 0.10500695]
 [0.03771429 0.14285714 0.29371429 0.33714286 0.03657143 0.152     ]
 [0.07332564 0.0386836  0.37471132 0.34988453 0.02424942 0.1391455 ]
 [0.0499232  0.05299539 0.25652842 0.47695853 0.00460829 0.15898618]
 [0.05417118 0.00758397 0.14842904 0.54279523 0.01625135 0.23076923]
 [0.03067093 0.12907348 0.28753994 0.46964856 0.01469649 0.06837061]
 [0.08295762 0.0613165  0.4220018  0.42560866 0.00811542 0.        ]
 [0.05393939 0.0830303  0.44909091 0.28545455 0.05636364 0.07212121]]

In [48]:
# Get phase duration percentage for each phase across all videos
# Output is a dictionary indexed by each phase. The value is the length of the key phase across all videos
def getPhasePercByPhase(phase_perc_mat_by_vid):
    return phase_perc_mat_by_vid.T
    

In [49]:
phase_perc_mat_by_phase = getPhasePercByPhase(phase_perc_mat_by_vid)
print(phase_perc_mat_by_phase)

[[0.01985659 0.13143872 0.06868538 0.01201771 0.02888752 0.08290536
  0.07301808 0.03771429 0.07332564 0.0499232  0.05417118 0.03067093
  0.08295762 0.05393939]
 [0.09321566 0.03641208 0.0620384  0.02972802 0.06207744 0.11225238
  0.03755216 0.14285714 0.0386836  0.05299539 0.00758397 0.12907348
  0.0613165  0.0830303 ]
 [0.32708218 0.29662522 0.17060561 0.53510436 0.24585126 0.25531915
  0.50625869 0.29371429 0.37471132 0.25652842 0.14842904 0.28753994
  0.4220018  0.44909091]
 [0.44070601 0.42273535 0.54357459 0.33965844 0.45482483 0.41599413
  0.2635605  0.33714286 0.34988453 0.47695853 0.54279523 0.46964856
  0.42560866 0.28545455]
 [0.01323773 0.0062167  0.01920236 0.01454775 0.04486785 0.03741746
  0.01460362 0.03657143 0.02424942 0.00460829 0.01625135 0.01469649
  0.00811542 0.05636364]
 [0.10590182 0.10657194 0.13589365 0.06894371 0.16349109 0.09611152
  0.10500695 0.152      0.1391455  0.15898618 0.23076923 0.06837061
  0.         0.07212121]]


In [50]:
# Get mean and std from each phase's length distribution
def getMeanAndStd(phase_perc_mat_by_phase):
    norm_params_dict = {}
    for phase, lengths in enumerate(phase_perc_mat_by_phase):
        mu, std = norm.fit(lengths)
        norm_params_dict[phase] = (mu, std)
    return norm_params_dict

In [51]:
norm_params_dict = getMeanAndStd(phase_perc_mat_by_phase)
for k, v in norm_params_dict.items():
    print("Duration percentage of phase {} has mean and std : {}". format(k, v))

Duration percentage of phase 0 has mean and std : (0.05710797112220632, 0.030321443261777936)
Duration percentage of phase 1 has mean and std : (0.06777261004314016, 0.03816546406393554)
Duration percentage of phase 2 has mean and std : (0.3263472992450161, 0.11284218205253618)
Duration percentage of phase 3 has mean and std : (0.4120390546260583, 0.08352923836196571)
Duration percentage of phase 4 has mean and std : (0.022210679714868137, 0.015080408295242551)
Duration percentage of phase 5 has mean and std : (0.11452238524871104, 0.053481343652044204)


In [52]:
# check the summation of mean percentage
0.05710797112220632 + 0.06777261004314016 + 0.3263472992450161 + 0.4120390546260583 + 0.022210679714868137 + 0.11452238524871104

1.0

## Transition Probability

In [53]:
def getTransProbMatWithZero(data_dict):
    # count transitions from phase i to all phases and devided by total transitions made from phase i
    trans_mat = np.array([[0.0]*6]*6)
    for vid in data_dict:
        label_list = data_dict[vid]["label list"]
        label_freq = data_dict[vid]["label freq"]
        total_trans_label_list = len(label_list)
        for index, phase in enumerate(label_list):
            # add freq-1 for each phase to the same phase transition
            # at the end of current label, add 1 to the transition 
            trans_mat[phase][phase] += label_freq[index] - 1
            if (index < total_trans_label_list - 1):
                trans_mat[phase][label_list[index+1]] += 1
#         print("Counting of transitions for {} looks like...".format(vid))
#         print(trans_mat)
    # calculate the probability of transition for each row
    for i in range(len(trans_mat)):
        total = np.sum(trans_mat[i])
        trans_mat[i] = trans_mat[i]/total
        
    print("Final transition probability matrix with phase zero looks like...")
    print(trans_mat)
    return trans_mat
    

In [54]:
np.set_printoptions(suppress=True, precision=6)
trans_mat = getTransProbMatWithZero(data_dict)

Final transition probability matrix with phase zero looks like...
[[0.941784 0.001878 0.00939  0.023474 0.011268 0.012207]
 [0.007553 0.989426 0.003021 0.       0.       0.      ]
 [0.002455 0.000153 0.997391 0.       0.       0.      ]
 [0.002773 0.       0.       0.996975 0.000252 0.      ]
 [0.031461 0.       0.       0.       0.968539 0.      ]
 [0.       0.       0.       0.       0.       1.      ]]


In [55]:
def getTransProbMatWithoutZero(data_dict):
    # The transition matrix here excludes phase zero
    # count transitions from phase i to all phases and devided by total transitions made from phase i
    trans_mat = np.array([[0.0]*5]*5)
    for vid in data_dict:
        label_list = data_dict[vid]["label list"]
        label_freq = data_dict[vid]["label freq"]
        total_trans_label_list = len(label_list)
        prev_phase = 0
        for index, phase in enumerate(label_list):
            # add freq-1 for each phase to the same phase transition
            # at the end of current label, add 1 to the transition 
            if phase == 0:
                continue
            if prev_phase != 0:
                trans_mat[prev_phase-1][phase-1] += 1
            trans_mat[phase-1][phase-1] += label_freq[index] - 1
            prev_phase = phase
#         print("Counting of transitions for {} looks like...".format(vid))
#         print(trans_mat)
    # calculate the probability of transition for each row
    for i in range(len(trans_mat)):
        total = np.sum(trans_mat[i])
        trans_mat[i] = trans_mat[i]/total
        
    print("Final transition probability matrix without phase zero looks like...")
    print(trans_mat)
    return trans_mat

In [56]:
np.set_printoptions(suppress=True, precision=6)
trans_mat_wo_zero = getTransProbMatWithoutZero(data_dict)

Final transition probability matrix without phase zero looks like...
[[0.989426 0.010574 0.       0.       0.      ]
 [0.00046  0.997391 0.002149 0.       0.      ]
 [0.       0.       0.996975 0.001765 0.00126 ]
 [0.       0.       0.024719 0.968539 0.006742]
 [0.       0.       0.       0.       1.      ]]


## Segment Transition Statistics
#### Segment is defined to be the collection of frames where the ground truth labels are the same 
#### Segment transition probability can be used together with frame transition matrix to fit in models like 
1. Hierarchical Hiddem Markov Model    
2. M/sM-Conditional Random Field   

In [57]:
# Function to compute segment transition probability with zero
def getSegmentTransMatrixWithZero(data_dict):
    trans_mat = np.array([[0.0]*6]*6)
    for vid in data_dict:
        label_list = data_dict[vid]["label list"]
        prev_label = -1
        for label in label_list:
            # current label is the first label: skip current round
            if prev_label == -1:
                prev_label = label
                continue
            # there is previous_label: compute the stats
            trans_mat[prev_label][label] += 1
            prev_label = label
    
    # calculate the probability of transition for each row
    for i in range(len(trans_mat)):
        total = np.sum(trans_mat[i])
        # if no transition to other phases, skip to avoid division error
        if total == 0: 
            continue
        trans_mat[i] = trans_mat[i]/total
    return trans_mat

In [58]:
np.set_printoptions(suppress=True, precision=6)
seg_trans_mat_w_zero = getSegmentTransMatrixWithZero(data_dict)
print("Segment transition statistics with phase zero look like...")
print(seg_trans_mat_w_zero)

Segment transition statistics with phase zero look like...
[[0.       0.032258 0.16129  0.403226 0.193548 0.209677]
 [0.714286 0.       0.285714 0.       0.       0.      ]
 [0.941176 0.058824 0.       0.       0.       0.      ]
 [0.916667 0.       0.       0.       0.083333 0.      ]
 [1.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.      ]]


In [59]:
# Function to compute segment transition probability without zero
def getSegmentTransMatrixWithoutZero(data_dict):
    trans_mat = np.array([[0.0]*5]*5)
    for vid in data_dict:
        label_list = data_dict[vid]["label list"]
        prev_label = -1
        for label in label_list:
            # skip phase zero
            if label == 0:
                continue
            # when there is previous_label: compute the stats
            if prev_label != -1:
                trans_mat[prev_label-1][label-1] += 1
            prev_label = label
    
    # print counts to check for correctness
    print("Counts of segments:")
    print(trans_mat)
    # calculate the probability of transition for each row
    for i in range(len(trans_mat)):
        total = np.sum(trans_mat[i])
        # if no transition to other phases, skip to avoid division error
        if total == 0: 
            continue
        trans_mat[i] = trans_mat[i]/total
    return trans_mat


In [60]:
np.set_printoptions(suppress=True, precision=6)
seg_trans_mat_wo_zero = getSegmentTransMatrixWithoutZero(data_dict)
print("Segment transition statistics without phase zero look like...")
print(seg_trans_mat_wo_zero)

Counts of segments:
[[ 0. 14.  0.  0.  0.]
 [ 3.  0. 14.  0.  0.]
 [ 0.  0.  0. 14. 10.]
 [ 0.  0. 11.  0.  3.]
 [ 0.  0.  0.  0.  0.]]
Segment transition statistics without phase zero look like...
[[0.       1.       0.       0.       0.      ]
 [0.176471 0.       0.823529 0.       0.      ]
 [0.       0.       0.       0.583333 0.416667]
 [0.       0.       0.785714 0.       0.214286]
 [0.       0.       0.       0.       0.      ]]


## Transition Probability incorporated with length of staying in current phase
Can be used to compare with only using fixed transitional probability matrix to fit into other models

In [74]:
# Function that incorporate time and transition matrix trained from ground truths to calculate the probability of transition at real time on prediction
# Prior knowledge: transitional prob matrix and mean and std of each phase length percentage
# Given input: length of stay in the current phrase 
# Posterior knowledge (returned value): updated transitional matrix
def getUpdatedMatrixWithPhaseLen(trans_prob_mat, norm_params_dict, cur_phase, prior_phase, prior_phase_fcount, total_frame):
    is_with_zero = len(trans_prob_mat) == 6
    loc, scale = norm_params_dict[prior_phase] # should take the parameters of the prior phase, which should be previous phase after updating
    # cur_phase_frame_count = cur_frame_num - prev_prior_ending_frame_num
    l = prior_phase_fcount/total_frame 
    cdf_l = norm.cdf(l, loc=loc, scale=scale)
    final_trans_prob = 0.0
    if (cur_phase == prior_phase):
        final_trans_prob = 1 - cdf_l
    else:        
        if is_with_zero:
            final_trans_prob = trans_prob_mat[prior_phase][cur_phase] * cdf_l
        else:
            final_trans_prob =  trans_prob_mat[prior_phase-1][cur_phase-1] * cdf_l
    
#     if prior_phase_fcount%10 == 0:
#         print("The frame count of current prior phase is {}".format(prior_phase_fcount))
#         print("cdf_l of current prior phase {} at {} of whole video is {}".format(prior_phase, l, cdf_l))
#         print("For the given prediction {}, the transition prob is {}".format(cur_phase, final_trans_prob))

    return final_trans_prob


# Times  (apart from mean and standard deviation of the percentage of duration of each phase)
- I don't think I should count the absolute number of frames for each phase in each video as the duration of each surgery is varies quite a lot?
1. At what portion of the surgery does the phase transition happen? (mean + std)
2. Length of the current phase (= total length so far - time when last phase transition occur, need to model dynamically)

# Fit priors into models
1. Simple Markov Chain with transitional probability matrix updated with time (inspired by the MDP paper)
2. Markov Renewal Process, which models the states and the jump time tuples
3. Continuous Time Markov Chain, which models with continuous time with Markov property
4. Conditional Random Field?

### How to fit the priors into PKI:
#### PKI = (init_phase, prior_th, prob_th)
1. Initialisation based on initialisation probability
2. Piror update -> number of phase to update prior: train using prediction labels st ill
3. Probability th -> when the prediction differs from prior, use pr(i|j, l) on prediction labels to check. If pr(i|j, l) > threshold, keep the predicted, else change to prior
> Q: no use of segment transition here?

In [75]:
'''
Prior Knowledge Inference:
Use prior knowledge of the phases to determine the current phase
Initialisation: based on initialisation probability
Piror_phase_th: still need to tune based on training data
Probability_th: use transitional probability factored by length cdf. 
            - If the transition has a confidence higher than the threshold, keep the CNN prediction state and update the prior (because that means the next phase has been reached)
            - else, udpate the prediction to prior
'''
def pkiWithPriors(seq_pred, prior_th, prob_th, trans_prob_mat, norm_params_dict, non_zero_init_prob_sorted, init_phases_sorted):
    # for logging
    trans_probs = []
    transit_history = []
    pred_output = np.array(seq_pred).copy()
    # initialise prior phase based on initialisation probability
    init_phase = -1
    rand_prob = random.random()
    print("Random init prob...")
    print(rand_prob)
    cur_prob_stack = 0
    for index, prob in enumerate(non_zero_init_prob_sorted):
        if rand_prob <= prob + cur_prob_stack:
            init_phase = init_phases_sorted[index]
            break
        cur_prob_stack += prob
    print("Initialising phase...")
    print(init_phase)
    
    total_frame = len(pred_output)
    prior = init_phase
    prev = pred_output[0]
    prev_phase_ending_frame = 0
    # keep a counter to count how many new phases have appeared. if count > 5, update the prior to new phases
    count_new_phases = 0
    for index, prediction in enumerate(pred_output):
        # if counter > 5, update prior
        if count_new_phases > prior_th and prior != prev:
            # update prior, i.e. the phase has changed into next phase
            prior = prev
            # update prev_phase_ending_frame
            prev_phase_ending_frame = index
        # if transitional probability from previous to current is smaller than threshold, update it to prior
        # otherwise, update prior to the new prediction, as the high transition probability indicates a transition in the phase is likely to happen

#         print("Index:{}".format(index))
#         print("Previous phase ending frame:{}".format(prev_phase_ending_frame))
        
        prior_phase_fcount = index - prev_phase_ending_frame
#         print("frames in current prior phase {} is {}".format(prior, prior_phase_fcount))
        
        trans_prob = getUpdatedMatrixWithPhaseLen(trans_prob_mat, norm_params_dict, prediction, prior, prior_phase_fcount, total_frame)
        # logging
        trans_probs.append(trans_prob)
        transit_history.append("{}->{}".format(prev, prediction))
        
        if trans_prob < prob_th:
            pred_output[index] = prior

        # logic to update counter in vanilla PKI
        # if current prediction is same as the previous one, counter increment by 1
        if prediction == prev:
            count_new_phases += 1
        else:
            # reset counter
            prev = prediction
            count_new_phases = 0
    
#     print("original predictions by CNN:")
#     print(seq_pred)
#     print("after applying pki with priors:")
#     print(pred_output)
#     print("transition history:")
#     print(transit_history)
#     print("transition probabilities:")
#     print(trans_probs)
    
    return pred_output




In [116]:
# Try pki with priors on example data, using trans_prob_mat without zero first (non_generative)
import random
pki_w_priors_output = pkiWithPriors(pred_labels, 4, 0.8, trans_mat_wo_zero, norm_params_dict, non_zero_init_prob_sorted, init_phases_sorted)


Random init prob...
0.044791989074050576
Initialising phase...
2


In [117]:
print("Accuracy of using only CNN for prediction is {}".format(accuracy_score(true_labels, pred_labels)))
print("Accuracy of using CNN followed by PKI with priors for prediction is {}".format(accuracy_score(true_labels, pki_w_priors_output)))

Accuracy of using only CNN for prediction is 0.6530612244897959
Accuracy of using CNN followed by PKI with priors for prediction is 0.7992277992277992


In [118]:
print(true_labels[:500])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [119]:
print(pred_labels[:500])

[2, 4, 2, 5, 4, 4, 2, 4, 4, 2, 2, 5, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 1, 1, 1, 1, 2, 1, 1, 1, 2, 4, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 5, 5, 4, 2, 3, 2, 2, 5, 5, 1, 2, 1, 1, 2, 5, 1, 1, 1, 5, 5, 5, 2, 1, 4, 4, 5, 5, 1, 4, 2, 2, 5, 1, 1, 5, 2, 5, 5, 5, 5, 5, 2, 5, 5, 2, 2, 1, 2, 2, 2, 5, 2, 1, 5, 5, 5, 5, 2, 2, 2, 2, 5, 2, 2, 2, 5, 2, 1, 5, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 1, 2, 2, 2, 2, 1, 1, 4, 1, 1, 2, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 5, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 5, 5, 2, 3, 2, 2, 2, 2, 4, 2, 5, 5, 5, 3, 5, 1, 5, 2, 2, 2, 4, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5, 2, 2, 5, 5, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 

In [120]:
print(pki_w_priors_output[:500])

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 